# Data Loading

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [ ]:
from sympy import Integral, Symbol

def get_gini(df, label):
    #클래스 빈도수 계산
    class_counts = df[label].value_counts()

    #데이터 총 개수
    total_instances = len(df)

    #지니 계수 계산
    sum_of_squares = sum((count / total_instances) ** 2 for count in class_counts)
    result = 1 - sum_of_squares

    return result

In [ ]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [ ]:
from itertools import combinations

def get_binary_split(df, attribute):
    uniques = list(df[attribute].unique())  #속성 데이터 고유값들을 담은 리스트
    result = []

    #가능한 모든 이진 분할 조합 생성
    for i in range(1, len(uniques)):
        for combo in combinations(uniques, i):
            #현재 조합과 나머지로 분할
            rest = set(uniques) - set(combo)
            result.append((set(combo), rest))

    return result

In [ ]:
get_binary_split(pd_data,'income')

[({'high'}, {'low', 'medium'}),
 ({'medium'}, {'high', 'low'}),
 ({'low'}, {'high', 'medium'}),
 ({'high', 'medium'}, {'low'}),
 ({'high', 'low'}, {'medium'}),
 ({'low', 'medium'}, {'high'})]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [ ]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    binary_splits = get_binary_split(df, attribute)
    n = len(df)

    for split in binary_splits:
        #분할된 두 그룹에 대한 데이터프레임 생성
        group1 = df[df[attribute].isin(split[0])]
        group2 = df[df[attribute].isin(split[1])]

        #각 그룹의 지니 계수 계산
        gini_group1 = get_gini(group1, label)
        gini_group2 = get_gini(group2, label)

        #가중 평균 지니 계수 계산
        weight_gini = (len(group1) / n) * gini_group1 + (len(group2) / n) * gini_group2

        #결과 저장 (분할을 튜플로 변환)
        #set은 직접적으로 키로 사용할 수 없으므로, 튜플로 변환하거나 문자열로 변환
        split_key = (tuple(split[0]), tuple(split[1]))  #또는 split_key = f"{tuple(split[0])} | {tuple(split[1])}" 로 문자열 형태로 저장 가능
        result[split_key] = weight_gini

    return result

In [ ]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{(('high',), ('medium', 'low')): 0.4428571428571429,
 (('medium',), ('high', 'low')): 0.4583333333333333,
 (('low',), ('high', 'medium')): 0.45,
 (('high', 'medium'), ('low',)): 0.45,
 (('high', 'low'), ('medium',)): 0.4583333333333333,
 (('medium', 'low'), ('high',)): 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [ ]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

((('high',), ('medium', 'low')), 0.4428571428571429)

In [ ]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

(('high',), ('medium', 'low'))

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [ ]:
#각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
label = pd_data.columns[-1]  #label 컬럼 얻기
features = list(pd_data.columns[:-1])  #label을 제외한 변수명 얻기

#각 변수를 대상으로 반복문 수행
for feature in features:
    gini_indices = get_attribute_gini_index(pd_data, feature, label)
    min_gini_index = min(gini_indices.values())  # 각 변수에 대한 최소 지니 지수 찾기
    print(f"Minimum Gini Index of {feature} : {min_gini_index:.4f}")

Minimum Gini Index of age : 0.3571
Minimum Gini Index of income : 0.4429
Minimum Gini Index of student : 0.3673
Minimum Gini Index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [ ]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{(('youth',), ('middle_aged', 'senior')): 0.3936507936507937,
 (('middle_aged',), ('youth', 'senior')): 0.35714285714285715,
 (('senior',), ('middle_aged', 'youth')): 0.45714285714285713,
 (('middle_aged', 'youth'), ('senior',)): 0.45714285714285713,
 (('senior', 'youth'), ('middle_aged',)): 0.35714285714285715,
 (('middle_aged', 'senior'), ('youth',)): 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [ ]:
from math import log2

def getEntropy(df, feature):
    #feature의 각 값에 대한 확률 분포를 계산함
    probabilities = df[feature].value_counts(normalize=True)

    #엔트로피 계산: 각 확률에 대해 p * log2(p)를 계산하고 합산
    #결과적으로 엔트로피에는 각 항의 음수를 취함
    entropy = -sum(probabilities * probabilities.apply(lambda p: log2(p) if p > 0 else 0))

    return entropy

In [ ]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [ ]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, target_feature):
    result = {}

    #목표 변수의 전체 엔트로피 계산
    total_entropy = getEntropy(df, target_feature)

    #목표 변수를 제외한 각 변수에 대해 반복
    for feature in df.columns:
        if feature != target_feature:
            #각 변수의 엔트로피 계산 및 결과 저장
            feature_entropy = getEntropy(df, feature)
            result[feature] = feature_entropy

    return result

In [ ]:
getGainA(pd_data, "class_buys_computer")

{'age': 1.5774062828523454,
 'income': 1.5566567074628228,
 'student': 1.0,
 'credit_rating': 0.9852281360342515}